# Migration of index of TheHive and Cortex created with ES5 to ES7

In [40]:
ES_HOST='http://localhost:9200'

## List available indices

In [61]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index           uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   new_cortex_4    npVQIiOfQaG1XL1AQxKm8g   5   0       8531            0     12.6mb         12.6mb
green  open   new_the_hive_15 2Y2fEs9fSii87NSxyVtTew   5   0      30998            0     26.1mb         26.1mb


In [62]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index           uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   new_cortex_4    npVQIiOfQaG1XL1AQxKm8g   5   0       8531            0     12.6mb         12.6mb
green  open   new_the_hive_15 2Y2fEs9fSii87NSxyVtTew   5   0      30998            0     26.1mb         26.1mb


If not, restore indexes.

## Setting up variables

In [42]:
THEHIVE_INDEX="the_hive_15"
CORTEX_INDEX="cortex_4"
NEW_THEHIVE_INDEX="new_the_hive_15"
NEW_CORTEX_INDEX="new_cortex_4"

## TheHive index

### Update the index ? 

Check wether you need to reindex or not.

In [43]:
curl -s ''${ES_HOST}/${THEHIVE_INDEX}'?human' | jq '.'${THEHIVE_INDEX}'.settings.index.version.created_string'

"5.6.13"


**If this start with '5.x' then proceed with next steps.** If result starts with '6.x' then, the index is ready for ES7.

### Create a new index

The First operation lies in creating a new index named `new_the_hive_15` with settings from current index `the_hive_15` (ensure to keep index version, needed for future upgrade).

In [44]:
curl -s -XPUT ''${ES_HOST}'/'${NEW_THEHIVE_INDEX}'' \
  -H 'Content-Type: application/json' \
  -d "$(curl -s ${ES_HOST}/${THEHIVE_INDEX} |\
   jq '.'${THEHIVE_INDEX}'|
   del(.settings.index.provided_name,
    .settings.index.creation_date,
    .settings.index.uuid,
    .settings.index.version,
    .settings.index.mapping.single_type,
    .mappings.doc._all)'
    )" | jq

{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "new_the_hive_15"
}


#### Check

Check the new index is well created:

In [45]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index           uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   new_the_hive_15 2Y2fEs9fSii87NSxyVtTew   5   0          0            0       460b           460b
green  open   the_hive_15     pT9kFs-oQP2J6E-PGxXJqg   5   0      30998           16     33.2mb         33.2mb
green  open   cortex_4        YzDbXlYAS0q15z14wmhWLQ   5   0       8531            8       13mb           13mb


### Proceed to reindex

Next operation lies in running the reindex command in the newly created index. **This action can take some time**:

In [46]:
curl -s -XPOST -H 'Content-Type: application/json' ${ES_HOST}/_reindex -d '{
  "conflicts": "proceed",
  "source": {
    "index": "'${THEHIVE_INDEX}'"
  },
  "dest": {
    "index": "'${NEW_THEHIVE_INDEX}'"
  }
}' | jq

{
  "took": 8370,
  "timed_out": false,
  "total": 5900,
  "updated": 0,
  "created": 5900,
  "deleted": 0,
  "batches": 6,
  "version_conflicts": 0,
  "noops": 0,
  "retries": {
    "bulk": 0,
    "search": 0
  },
  "throttled_millis": 0,
  "requests_per_second": -1,
  "throttled_until_millis": 0,
  "failures": []
}


#### Check 

Run the following command, and ensure the new index is like the current one (size can vary):

In [47]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index           uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   new_the_hive_15 2Y2fEs9fSii87NSxyVtTew   5   0      30998            0     26.1mb         26.1mb
green  open   the_hive_15     pT9kFs-oQP2J6E-PGxXJqg   5   0      30998           16     33.2mb         33.2mb
green  open   cortex_4        YzDbXlYAS0q15z14wmhWLQ   5   0       8531            8       13mb           13mb



### Delete old indices

This is the thrilling part. 
Now the new index `new_the_hive_15` is created and similar the_hive_15,  older indexes **should be completely deleted** from the database. To delete index named `the_hive_15`, run the following command:  


Run the same command for older indexes if exist (the_hive_14, the_hive_13....). Elasticsearch 7.x cannot run with index created with Elasticsearch 5.x.



In [48]:
curl -s -XDELETE ${ES_HOST}/$THEHIVE_INDEX | jq

{
  "acknowledged": true
}


### Create an alias 

Before stopping Elasticsearch service, let’s create an alias to keep index names in the future.  

Doing so will allow TheHive 3.5.0 to find the index without updating the configuration file. 

In [49]:
curl -s -XPOST -H 'Content-Type: application/json'  ''${ES_HOST}'/_aliases' -d '{
    "actions": [
        {
            "add": {
                "index": "'${NEW_THEHIVE_INDEX}'",
                "alias": "'${THEHIVE_INDEX}'"
            }
        }
    ]
}' | jq

{
  "acknowledged": true
}


Check the alias has been well created by running the following command.

In [50]:
curl -s -XGET ${ES_HOST}/_alias?pretty 

{
  "cortex_4" : {
    "aliases" : { }
  },
  "new_the_hive_15" : {
    "aliases" : {
      "the_hive_15" : { }
    }
  }
}


### Check the alias is reachable and your index is ready for ES 7

In [51]:
curl -s ''${ES_HOST}/${THEHIVE_INDEX}'?human' | jq '.'${NEW_THEHIVE_INDEX}'.settings.index.version.created_string'

"6.8.13"


**TheHive is ready to be used with the restored data**

## Cortex index

### Update the index ? 

In [52]:
curl -s ''${ES_HOST}/${CORTEX_INDEX}'?human' | jq '.'${CORTEX_INDEX}'.settings.index.version.created_string'

"5.6.13"


**If this start with '5.x' then proceed with next steps.** If result starts with '6.x' then, the index is ready for ES7.

### Create a new index

The First operation lies in creating a new index named `new_cortex_4` with settings from current index `cortex_4` (ensure to keep index version, needed for future upgrade).

In [53]:
curl -s -XPUT ''${ES_HOST}/${NEW_CORTEX_INDEX}'' \
  -H 'Content-Type: application/json' \
  -d "$(curl -s ${ES_HOST}/${CORTEX_INDEX} |\
   jq '.'${CORTEX_INDEX}' |
   del(.settings.index.provided_name,
    .settings.index.creation_date,
    .settings.index.uuid,
    .settings.index.version,
    .settings.index.mapping.single_type,
    .mappings.doc._all)'
    )" | jq

{
  "acknowledged": true,
  "shards_acknowledged": true,
  "index": "new_cortex_4"
}


#### Check

Check the new index is well created:

In [54]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index           uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   new_cortex_4    npVQIiOfQaG1XL1AQxKm8g   5   0          0            0       401b           401b
green  open   new_the_hive_15 2Y2fEs9fSii87NSxyVtTew   5   0      30998            0     26.1mb         26.1mb
green  open   cortex_4        YzDbXlYAS0q15z14wmhWLQ   5   0       8531            8       13mb           13mb


### Proceed to reindex

Next operation lies in running the reindex command in the newly created index:

In [55]:
curl -s -XPOST -H 'Content-Type: application/json' ${ES_HOST}/_reindex -d '{
  "conflicts": "proceed",
  "source": {
    "index": "'${CORTEX_INDEX}'"
  },
  "dest": {
    "index": "'${NEW_CORTEX_INDEX}'"
  }
}' | jq

{
  "took": 2039,
  "timed_out": false,
  "total": 8155,
  "updated": 0,
  "created": 8155,
  "deleted": 0,
  "batches": 9,
  "version_conflicts": 0,
  "noops": 0,
  "retries": {
    "bulk": 0,
    "search": 0
  },
  "throttled_millis": 0,
  "requests_per_second": -1,
  "throttled_until_millis": 0,
  "failures": []
}


#### Check 

Run the following command, and ensure the new index is like the current one (size can vary):

In [56]:
curl -s ${ES_HOST}/_cat/indices\?v

health status index           uuid                   pri rep docs.count docs.deleted store.size pri.store.size
green  open   new_cortex_4    npVQIiOfQaG1XL1AQxKm8g   5   0       8531            0     12.6mb         12.6mb
green  open   new_the_hive_15 2Y2fEs9fSii87NSxyVtTew   5   0      30998            0     26.1mb         26.1mb
green  open   cortex_4        YzDbXlYAS0q15z14wmhWLQ   5   0       8531            8       13mb           13mb



### Delete old indices

This is the thrilling part. 
Now the new index `new_cortex_4` is created and similar cortex_4,  older indexes **should be completely deleted** from the database. To delete index named `cortex_4`, run the following command:  


Run the same command for older indexes if exist (cortex_1, cortex_2....). Elasticsearch 7.x cannot run with index created with Elasticsearch 5.x.



In [57]:
curl -s -XDELETE ${ES_HOST}/$CORTEX_INDEX | jq

{
  "acknowledged": true
}


### Create an alias 

Before stopping Elasticsearch service, let’s create an alias to keep index names in the future.  

Doing so will allow Cortex 3.1.0 to find the index without updating the configuration file. 

In [58]:
curl -s -XPOST -H 'Content-Type: application/json'  ''${ES_HOST}'/_aliases' -d '{
    "actions": [
        {
            "add": {
                "index": "'${NEW_CORTEX_INDEX}'",
                "alias": "'${CORTEX_INDEX}'"
            }
        }
    ]
}' | jq

{
  "acknowledged": true
}


Check the alias has been well created by running the following command:

In [59]:
curl -s -XGET ${ES_HOST}/_alias?pretty 

{
  "new_cortex_4" : {
    "aliases" : {
      "cortex_4" : { }
    }
  },
  "new_the_hive_15" : {
    "aliases" : {
      "the_hive_15" : { }
    }
  }
}


### Check the alias is reachable and version of index is 6.x

In [60]:
curl -s ''${ES_HOST}/${CORTEX_INDEX}'?human' | jq '.'${NEW_CORTEX_INDEX}'.settings.index.version.created_string'

"6.8.13"


**Cortex is ready to be used with the restored data**